In [2]:
import sys
import os
sys.path.insert(1, '/home/jovyan/graphormer_v2/')

import torch
import numpy as np
import torch_geometric.datasets
from ogb.graphproppred import PygGraphPropPredDataset
from ogb.lsc.pcqm4m_pyg import PygPCQM4MDataset
import pyximport
from torch_geometric.data import InMemoryDataset, download_url
import pandas as pd
from sklearn import preprocessing

pyximport.install(setup_args={'include_dirs': np.get_include()})
import os.path as osp
from torch_geometric.data import Data
import time

from torch_geometric.utils import add_self_loops, negative_sampling
from graphormer.data.wrapper import preprocess_item

In [82]:
from graphormer.data.pyg_datasets.pyg_dataset import GraphormerPYGDataset
from graphormer.data.dataset import GraphormerDataset

In [48]:
class single_geo_Omsk(InMemoryDataset):
    
    
    def __init__(self, root, transform=None, pre_transform=None, split = 'train'):
        super(single_geo_Omsk, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_dir + '/predict_data.pt')
        # self.data = torch.load(self.processed_dir + '/' + f'{split}.pt')
#         self.raw_dir = '/home/jovyan/'
        
    @property
    def raw_dir(self) -> str:
        return '/home/jovyan/tte_data/'
    
    
    @property
    def raw_file_names(self):
        return ['omsk_full_routes_final_weather_L_NaN_filtered_FIXED.csv']

    @property
    def processed_file_names(self):
        return ['predict_data.pt']
    
    @property
    def processed_dir(self):
        return osp.join(self.root)

    def download(self):
        path = download_url(self.url, self.raw_dir)
        print(self.processed_paths[0])
    
    def my_load_dataset(self):
        return [self.data, self.slices]
    
    def process(self):
        
        # Read data
        start_time = time.time()
        data = pd.read_csv(osp.join('/home/jovyan/tte_data/', 'omsk_full_routes_final_weather_L_NaN_filtered_FIXED.csv'))
        data = data[data['rebuildCount']<=1].reset_index(drop = True).copy()
        # shape = int(data.shape[0]÷)
        shape = int(10)
        data = data[0:shape].copy()
        
        data = data.drop(columns = ['Unnamed: 0'])
        data['hour'] = data['start_timestamp'].apply(lambda x: int(x[-10:-8]))
        # Graph 
        graph_columns_gran = ['edges', 'time', 'speed', 'length']
        edges = ['edges']
        target = ['time']
        node_features_gran = ['speed', 'length']

        edge_features_agg = [' start_point_part', 'finish_point_part', 'day_period', 'week_period', 'clouds', 'snow', 'temperature', 'wind_dir', 'wind_speed', 'pressure','hour']

        
        all_speed = []
        all_length = []
        for i in range(0,shape):
            data_row = data[i:i+1].reset_index(drop = True).copy()
            speed_list = [int(x) for x in (data_row['speed'].values[0].replace("'",'').split(','))]
            list_length = [int(x) for x in (data_row['length'].values[0].replace("'",'').split(','))]
            all_speed.append(speed_list)
            all_length.append(list_length)
            
        all_speed = [item for sublist in all_speed for item in sublist]
        all_length = [item for sublist in all_length for item in sublist]
    
        data_split_dict = dict()
        data_split_dict['all'] = np.arange(0, int(data.shape[0]))
        
        data_list = []
        for i in data_split_dict['all']:
            data_row = data.iloc[[i],].reset_index(drop = True).copy()

            edge_list = [int(x) for x in (data_row['edges'].values[0].replace("'",'').split(','))]
            speed_list = [int(x) for x in (data_row['speed'].values[0].replace("'",'').split(','))]
            list_length = [int(x) for x in (data_row['length'].values[0].replace("'",'').split(','))]

            source = edge_list.copy()
            target = edge_list[1:].copy() + [edge_list[0]].copy()

            data_row_gran = pd.DataFrame()
            data_row_gran['source'] = source
            data_row_gran['target'] = target
            data_row_gran['speed'] = speed_list
            data_row_gran['length'] = list_length


            target_val = data_row['RTA'].values[0]


            data_row_gran['speed'] = data_row_gran['speed']/np.mean(speed_list)
            data_row_gran['length'] = data_row_gran['length']/np.mean(list_length)

            for col in edge_features_agg:
                data_row_gran[col] = data_row[col].values[0]

            total_nodes_list = list(set(list(data_row_gran.source.values)))
            le = preprocessing.LabelEncoder()
            le.fit(total_nodes_list)

            data_row_gran['source'] = le.transform(data_row_gran.source.values)
            data_row_gran['target'] = le.transform(data_row_gran.target.values)

            total_nodes_list = list(set(list(data_row_gran.source.values)))

            edge_index = torch.tensor(torch.from_numpy(data_row_gran[['source','target']].values.T),dtype = torch.long)


            # Define tensor of nodes features
            x = torch.tensor(torch.from_numpy(data_row_gran[['speed','length'] + edge_features_agg].values),dtype = torch.long)


            # Define tensor of edge features
            edge_num_feach = 1
            edge_attr = torch.from_numpy(np.ones(shape = ((edge_index.size()[1]), edge_num_feach)))
            edge_attr = torch.tensor(edge_attr,dtype = torch.long)

            # Define tensor of targets
            y = torch.tensor(target_val,dtype = torch.long)


            data_graph = Data(x=x, edge_index = edge_index, edge_attr = edge_attr, y=y)
            data_list.append(data_graph)
        torch.save(self.collate(data_list), osp.join(self.processed_dir, 'predict_data.pt'))

In [49]:
root = osp.join('dataset', 'omsk')
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
data = single_geo_Omsk(root = raw_dir)

Processing...
Done!


/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:116: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [51]:
data[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])

In [37]:
data[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])

In [64]:
preprocess_item(data[0])

start find path dijkstra
dijkstra end with time 0.022580623626708984
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.010469436645507812


Data(attn_bias=[54, 54], attn_edge_type=[53, 53, 1], edge_attr=[53, 1], edge_index=[2, 53], edge_input=[53, 53, 52, 1], in_degree=[53], out_degree=[53], spatial_pos=[53, 53], x=[53, 13], y=[1])

In [40]:
preprocess_item(data[1])

start find path dijkstra
dijkstra end with time 0.038013458251953125
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.0209200382232666


Data(attn_bias=[73, 73], attn_edge_type=[72, 72, 1], edge_attr=[72, 1], edge_index=[2, 72], edge_input=[72, 72, 71, 1], in_degree=[72], out_degree=[72], spatial_pos=[72, 72], x=[72, 13], y=[1])

In [44]:
ll = torch.load('dataset/omsk/processed/data_omsk_1/train.pt')

In [47]:
ll[1]['x']

tensor([       0,       53,      125,  ..., 24401063, 24401090, 24401141])

In [ ]:
single_geo_Omsk
GraphormerPYGDataset
GraphormerDataset
batched_data = self.dm.dataset_predict
batched_data = BatchedDataDataset(
        batched_data,
        max_node=self.max_nodes(),
        multi_hop_max_dist=self.cfg.multi_hop_max_dist,
        spatial_pos_max=self.cfg.spatial_pos_max,
    )

    data_sizes = np.array([self.max_nodes()] * len(batched_data))

    target = TargetDataset(batched_data)
    # print('target 1 size', target.size())

    dataset = NestedDictionaryDataset(
        {
            "nsamples": NumSamplesDataset(),
            "net_input": {"batched_data": batched_data},
            "target": target,
        },
        sizes=data_sizes,
    )

    if split == "train" and self.cfg.train_epoch_shuffle:
        dataset = EpochShuffleDataset(
            dataset, size=len(dataset), seed=self.cfg.seed
        )

    logger.info("Loaded {0} with #samples: {1}".format(split, len(dataset)))

    self.datasets[split] = dataset
    self.batched_data = batched_data
    return self.datasets[split]

In [104]:
from graphormer.data.pyg_datasets.pyg_dataset import GraphormerPYGDataset
name = 'omsk'
root = osp.join('dataset', name)
raw_dir = osp.join(root, 'processed', 'data_omsk_1')
data = single_geo_Omsk(root = raw_dir)
proc_data = preprocess_item(data[0])
GPYG = GraphormerPYGDataset_predict(None,seed,None,preprocess_item(data[0]),'omsk')
dataset = NestedDictionaryDataset(
            {
                "nsamples": NumSamplesDataset(),
                "net_input": {"batched_data": GPYG},
                "target": batched_data,
            }
        )

start find path dijkstra
dijkstra end with time 0.016960620880126953
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.010648012161254883
start find path dijkstra
dijkstra end with time 0.01824355125427246
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.010607242584228516


In [105]:
dataset

In [58]:
proc_data = preprocess_item(data[0])

start find path dijkstra
dijkstra end with time 0.016582489013671875
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.010667562484741211


In [61]:
seed = 71
name = 'omsk'
GraphormerPYGDataset(
                    None,
                    seed,
                    None,
                    None,
                    None,
                    True,
                    True,
                    True,
                    name)

TypeError: object of type 'bool' has no len()

In [136]:
from torch_geometric.data import Dataset
from functools import lru_cache


class GraphormerPYGDataset_predict(Dataset):
    def __init__(
        self,
        dataset: Dataset,
        seed: int = 0,
        predict_idx=None,
        predict_set=None,
        name = None
    ):
        self.name = name
        self.dataset = dataset
        if self.dataset is not None:
            self.num_data = len(self.dataset)
        self.seed = seed

        self.num_data = len(predict_set) 
        self.predict_idx = predict_idx
        self.predict_data = self.create_subset(predict_set)
        self.__indices__ = None

    def index_select(self, idx):
        dataset = copy.copy(self)
        dataset.dataset = self.dataset.index_select(idx)
        if isinstance(idx, torch.Tensor):
            dataset.num_data = idx.size(0)
        else:
            dataset.num_data = idx.shape[0]
        dataset.__indices__ = idx
        dataset.predict_data = None
        dataset.predict_idx = None
        return dataset

    def create_subset(self, subset):
        dataset = copy.copy(self)
        dataset.dataset = subset
        dataset.num_data = len(subset)
        dataset.__indices__ = None
        dataset.predict_data = None
        dataset.predict_idx = None
        return dataset

    @lru_cache(maxsize=16)
    def __getitem__(self, idx):
        if isinstance(idx, int):
            print('idx:', idx)
            print('self.dataset:', self.dataset)
            item = self.dataset[idx]
            item.idx = idx
            item.y = item.y.reshape(-1)
            return preprocess_item(item)
        else:
            raise TypeError("index to a GraphormerPYGDataset can only be an integer.")

    def __len__(self):
        return self.num_data

In [139]:
import copy


GPYG = GraphormerPYGDataset_predict(
                    data,
                    seed,
                    None,
                    data,
                    'omsk'
                )

In [142]:
GPYG[0]

idx: 0
self.dataset: single_geo_Omsk(10)
start find path dijkstra
dijkstra end with time 0.01664423942565918
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.010808944702148438


Data(attn_bias=[54, 54], attn_edge_type=[53, 53, 1], edge_attr=[53, 1], edge_index=[2, 53], edge_input=[53, 53, 52, 1], idx=0, in_degree=[53], out_degree=[53], spatial_pos=[53, 53], x=[53, 13], y=[1])

In [134]:
data[0]

Data(edge_attr=[53, 1], edge_index=[2, 53], x=[53, 13], y=[1])

In [87]:
GraphormerPYGDataset_predict

__main__.GraphormerPYGDataset_predict

In [93]:
GraphormerPYGDataset_predict

__main__.GraphormerPYGDataset_predict

In [92]:
GPYG

GraphormerPYGDataset_predict(10)

In [94]:
batched_data = GPYG

In [95]:
batched_data = BatchedDataDataset(
        batched_data,
        max_node=self.max_nodes(),
        multi_hop_max_dist=self.cfg.multi_hop_max_dist,
        spatial_pos_max=self.cfg.spatial_pos_max,
    )

NameError: name 'BatchedDataDataset' is not defined

In [97]:
from fairseq.data import (
    NestedDictionaryDataset,
    NumSamplesDataset,
)

In [112]:
from graphormer.data.pyg_datasets.pyg_dataset import GraphormerPYGDataset
from graphormer.data.dataset import (
    BatchedDataDataset,
    TargetDataset,
    GraphormerDataset)

batched_data = BatchedDataDataset(
            batched_data
        )

In [113]:
batched_data

In [114]:

dataset = NestedDictionaryDataset(
            {
                "nsamples": NumSamplesDataset(),
                "net_input": {"batched_data": batched_data},
                "target": batched_data,
            }
        )

In [102]:
sample = dataset

In [103]:
# infer
y_pred = []
with torch.no_grad():
    model.eval()
    sample = utils.move_to_cuda(sample)
    y = model(**sample["net_input"])[:, 0, :].reshape(-1)
    y_pred.extend(y.detach().cpu())
    torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [117]:
batch_iterator = task.get_batch_iterator(dataset=GPYG)
itr = batch_iterator.next_epoch_itr(shuffle=False, set_dataset_epoch=False)
progress = progress_bar.progress_bar(itr)

NameError: name 'task' is not defined

In [119]:
GPYG[1]

TypeError: 'NoneType' object is not subscriptable